In [0]:
import requests
import json
import phonenumbers 
import re
import datetime

from pyspark.sql.types import StringType, IntegerType, TimestampType, DateType, DoubleType, StructType, StructField
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col

import time


In [0]:
# Create Spark Context

sparkContext = SparkSession \
    .builder \
    .appName("Apartment POC") \
    .getOrCreate()

# Create Hive Context
hivecontext=HiveContext(sparkContext)

# configure Hive Context
hivecontext.setConf('hive.support.concurrency','true');
hivecontext.setConf('hive.enforce.bucketing','true');
hivecontext.setConf('hive.exec.dynamic.partition.mode','nostrict');
hivecontext.setConf('hive.compactor.initiator.on','true');
hivecontext.setConf('hive.compactor.worker.threads','1');

sqlContext = SQLContext(sparkContext)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2320188403031192> in <module> 
 7 
 8 # Create Hive Context 
 ----> 9 hivecontext = HiveContext ( sparkContext ) 
 10 
 11 # configure Hive Context 

 /databricks/spark/python/pyspark/sql/context.py in __init__ (self, sparkContext, jhiveContext) 
 486 DeprecationWarning)
 487 if jhiveContext is None : 
 --> 488 sparkContext . _conf . set ( "spark.sql.catalogImplementation" , "hive" ) 
 489 sparkSession = SparkSession . builder . _sparkContext ( sparkContext ) . getOrCreate ( ) 
 490 else : 

 AttributeError : 'SparkSession' object has no attribute '_conf'

In [0]:
# Schema for Agencies JSON
agency_schema = StructType([
            StructField("Agencies_ID", IntegerType(), False),
            StructField("Agencies_Name", StringType(), True),
            StructField("Driver_ID", IntegerType(), False),
            StructField("Driver-Name", StringType(), False),
            StructField("Mobile", StringType(), False),
            StructField("State", StringType(), True),
            StructField("City", StringType(), True), 
            StructField("Address", StringType(), True), 
            StructField("Vehicle_id", IntegerType(), True)])

# Food category schema
foodcategory_schema = StructType([
            StructField("C_ID", IntegerType(), False),
            StructField("C_Name", StringType(), False)])

# Food Sub Category Schema
food_subcategory_schema = StructType([
            StructField("SC_ID", IntegerType(), False),
            StructField("SC_Name", StringType(), True),
            StructField("C_ID", IntegerType(), False) ])

# Customer Schema
customer_schema = StructType([
            StructField("Name", StringType(), True),
            StructField("Email", StringType(), True),
            StructField("Contact_No", StringType(), True),
            StructField("Passport_id", StringType(), True)])

# Schema for Vehicle
vehicle_schema = StructType([
            StructField("Vehicle_type", StringType(), False),
            StructField("Vehicle_ID", IntegerType(), True)])

 

# Schema for Delivery 
delivery_schema = StructType([
            StructField("Driver_ID", IntegerType(), False),
            StructField("Destination_State", StringType(), True),
            StructField("Destination_City", StringType(), False),
            StructField("Destination_Address", StringType(), True),
            StructField("Postal_Code", StringType(), True),
            StructField("Departure_Time", StringType(), True),
            StructField("Delivery_Time", StringType(), True),
            StructField("Order_ID", IntegerType(), False)])

# Schema  for Order Details
order_details_schema = StructType([
            StructField("Order_no", IntegerType(), False),
            StructField("Food_Id", IntegerType(), False),
            StructField("Portion_size", StringType(), False),
            StructField("Qty", IntegerType(), True),
            StructField("Amount", StringType(), True) ])

# Schema for Order
order_schema = StructType([
            StructField("Order_No", IntegerType(), False),
            StructField("Ticket_No", IntegerType(), True),
            StructField("Driver_ID", IntegerType(), False),
            StructField("Date", StringType(), True),
            StructField("Time", StringType(), True), 
            StructField("order_Type", StringType(), True), 
            StructField("Delivery_Time_(Minutes)", IntegerType(), True)])



In [0]:
# Function to get SparkDataFrame after reading JSON data from API
def getSparkDataFrame(url, schema):
  appdf = requests.get(url)
  objJSON = appdf.json()
  a=[json.dumps(objJSON)]
  jsonRDD = sc.parallelize(a)
  df = spark.readStream.schema(schema).json(jsonRDD)
  return df

def getJSONString(url):
  appdf = requests.get(url)
  str=''
  for line in appdf.iter_lines():
    str = line.decode(encoding='utf-8', errors='strict')
    # escaping \n works for python3, if it's python 2 no need to escape
    str = str.replace('\\n', '')
  json_str=json.loads(str)
  return json_str

In [0]:

#df = getSparkDataFrame("https://my.api.mockaroo.com/Agencies.json?key=6af9c3e0", agency_schema)
 
# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
df = (
  spark
    .readStream                       
    .schema(agency_schema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(getJSONString("https://my.api.mockaroo.com/Agencies.json?key=6af9c3e0"))
)
df.printSchema()
df.show(5)

--------------------------------------------------------------------------- 
 JSONDecodeError Traceback (most recent call last)
 <command-4398120066749158> in <module> 
 8 . schema ( agency_schema ) # Set the schema of the JSON data 
 9 . option ( "maxFilesPerTrigger" , 1 ) # Treat a sequence of files as a stream by picking one file at a time 
 ---> 10 . json ( getJSONString ( "https://my.api.mockaroo.com/Agencies.json?key=6af9c3e0" ) ) 
 11 )
 12 df . printSchema ( ) 

 <command-4398120066749157> in getJSONString (url) 
 15 # escaping \n works for python3, if it's python 2 no need to escape 
 16 str = str . replace ( '\\n' , '' ) 
 ---> 17 json_str = json . loads ( str ) 
 18 return json_str

 /usr/lib/python3.5/json/__init__.py in loads (s, encoding, cls, object_hook, parse_float, parse_int, parse_constant, object_pairs_hook, **kw) 
 317 parse_int is None and parse_float is None and 
 318 parse_constant is None and object_pairs_hook is None and not kw):
 --> 319 return _default_decoder . decode ( s ) 
 320 if cls is None : 
 321 cls = JSONDecoder

 /usr/lib/python3.5/json/decoder.py in decode (self, s, _w) 
 340 end = _w ( s , end ) . end ( ) 
 341 if end != len ( s ) : 
 --> 342 raise JSONDecodeError ( "Extra data" , s , end ) 
 343 return obj
 344 

 JSONDecodeError : Extra data: line 1 column 5 (char 4)

In [0]:
timestamp = datetime.datetime.fromtimestamp(time.time())
dateTimeStr = datetime.datetime.today().strftime("%m-%d-%Y %H:%M:%S")

df = df.withColumn("EventTimestamp", lit(timestamp))
df = df.withColumn("Load_date", lit(dateTimeStr))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4398120066749159> in <module> 
 2 dateTimeStr = datetime . datetime . today ( ) . strftime ( "%m-%d-%Y %H:%M:%S" ) 
 3 
 ----> 4 df = df . withColumn ( "EventTimestamp" , lit ( timestamp ) ) 
 5 df = df . withColumn ( "Load_date" , lit ( dateTimeStr ) ) 

 NameError : name 'df' is not defined

In [0]:
import sched, time
s = sched.scheduler(time.time, time.sleep)
def callJSON(sc): 
    print("Calling JSON API...")
    # get the spark dataframe
    
    s.enter(60, 1, callJSON, (s,))

 
s.enter(60, 1, callJSON, (s,))
s.run()

java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:717)
	at sun.nio.ch.SocketAdaptor.connect(SocketAdaptor.java:111)
	at org.eclipse.jetty8.client.SelectConnector.startConnection(SelectConnector.java:82)
	at org.eclipse.jetty8.client.HttpDestination.startNewConnection(HttpDestination.java:292)
	at org.eclipse.jetty8.client.HttpDestination.doSend(HttpDestination.java:597)
	at org.eclipse.jetty8.client.HttpDestination.send(HttpDestination.java:532)
	at org.eclipse.jetty8.client.HttpClient.send(HttpClient.java:180)
	at com.databricks.rpc.JettyClient.$anonfun$sendRawExchange$2(JettyClient.scala:328)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.rpc.JettyClient.withAttributionContext(JettyClient.scala:33)
	at com.databricks.rpc.JettyClient.sendRawExchange(JettyClient.scala:318)
	at com.databricks.rpc.JettyClient.sendExchange(JettyClient.scala:243)
	at com.databricks.rpc.JettyClient.sendAsync(JettyClient.scala:152)
	at com.databricks.rpc.BoundRPCClient.sendAsync(BoundRPCClient.scala:54)
	at com.databricks.backend.daemon.driver.DriverClient.com$databricks$backend$daemon$driver$DriverClient$$send(DriverClient.scala:95)
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$1(DriverClient.scala:283)
	at com.databricks.util.FutureUtils$.retry(FutureUtils.scala:87)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:91)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:89)
	at scala.concurrent.Future.$anonfun$recoverWith$1(Future.scala:417)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:316)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:257)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:316)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)